# Example usage of `wqxweblib`

For this example, we're going to store our credentials in an environment variable loaded from a file called `.env`. In production, you might use a different mechanism for storing and loading credentials, but for our purposes this will work fine. 

Before running any of this code, create a file called `.env` in this project's directory with your user id and private key from WQX Web in CDX. If you're not sure what that is, reach out to the person who gave you credentials to WQX Web and ask how to obtain a private key for API access to WQX Web.

The `.env` file should be key value pairs like this:

    WQXWEB_USERID=someuser
    WQXWEB_PRIVATEKEY=rn9+r8E7t2ePfTqjkcocJcGU9BAl/x+X1RSf1miOaB1tLo1fi448sPzIeJCg5rMnvgVtDFfOsrpOsNVQ7/q3WA==

After you've created that, you can use the following code block to load those environment variables into Python.

In [1]:
# Install dotenv
!python3 -m pip install --user --upgrade --no-warn-script-location python-dotenv
# Import libraries
from dotenv import load_dotenv
# Load environment variables from file
load_dotenv(".env")
# Assign the user id and private key to their own variables
user_id = os.environ.get("WQXWEB_USERID")
private_key = os.environ.get("WQXWEB_PRIVATEKEY")

print(f"You've loaded the user id {user_id} with{'out' if private_key is None else ''} a private key.")

You've loaded the user id someuser with a private key.


Install `wqxweblib` using `pip3` if it's not already installed.

In [2]:
!python3 -m pip install --user --upgrade wqxweblib

Import the `WQXWeb` class from the `wqxweblib` library and create an object from it using your EPA-provided authentication credentials.

This will be the primary way you interact with WQX Web.

In [3]:
from wqxweblib import WQXWeb

wqxweb = WQXWeb(userID=user_id, privateKey=private_key)

print("That worked so far, but in practice you should usually trap errors in a try/catch block.")

That worked so far, but in practice you should usually trap errors in a try/catch block.


We don't need to use real data for this simple example, so we're going to use the sample data that happens to be included with the Physical Chemical Template which is associated with Import Configuration `7411`. We'll download it to the current directory as `results.xlsx`.

Normally, you would want to gather the real data into a CSV file or XLSX file from sensors or other sources.

In [4]:
!wget -O results.xlsx ftp://newftp.epa.gov/storet/wqx/wqx_web_templates/wqxweb3_physical_chemical_package/Physical%20Chemical%20Template.xlsx

--2021-03-03 15:07:47--  ftp://newftp.epa.gov/storet/wqx/wqx_web_templates/wqxweb3_physical_chemical_package/Physical%20Chemical%20Template.xlsx
           => ‘results.xlsx’
Resolving newftp.epa.gov (newftp.epa.gov)... 134.67.100.58, 134.67.100.58
Connecting to newftp.epa.gov (newftp.epa.gov)|134.67.100.58|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /storet/wqx/wqx_web_templates/wqxweb3_physical_chemical_package ... done.
==> SIZE Physical Chemical Template.xlsx ... 666851
==> PASV ... done.    ==> RETR Physical Chemical Template.xlsx ... done.
Length: 666851 (651K) (unauthoritative)

Physical Chemical T 100%[===================>] 651.22K  1.51MB/s    in 0.4s    

2021-03-03 15:07:48 (1.51 MB/s) - ‘results.xlsx’ saved [666851]



Now that we have some data, let's upload it. The `Upload` function does what we need and returns a `fileId` which we will use later to identify the uploaded file.

In [5]:
with open("results.xlsx", mode='rb') as file:
    file_id = wqxweb.Upload(filename="results.xlsx", contents=file.read())

print(f"The file uploaded successfully with file_id {file_id}")

The file uploaded successfully with file_id 4a327079-6a3e-4ea6-98e0-1e558140ebce


If we had attachments for the data, we would use the `UploadAttachment` function to upload it.

One example of when you might do this is if you have a continuous data set which you aggregate for the primary upload and want to attach the raw data along with it. That way the aggregated data can be consumed via WQP and researchers can still download and explore the more detailed data that you derived the submission from.

We're not going to do that in this example, so we'll move right on to telling WQX Web to start importing the data we just uploaded. At this point, the file is just sitting in temporary storage waiting for further instructions.

The import will be triggered when we call `StartImport`. It accepts many arguments, but the ones we're using are:

 - `importConfigurationId` - The import configuration ID can be found on the "Import Configurations" screen of the WQX Web interface. In this case, we're using import configuration ID `7411` which corresponds to the one named ".Template Physical/Chemical (Template)" and by default uses the WQXTEST organization.
 - `fileId` - The file ID is the string which was returned from the `Upload` function earlier.
 - `fileType` - We're using an XLSX file which is one of the few allowed types. To specify it, we can use the constant `WQXWeb.XLSX`
 - `worksheetsToImport` - Since we're uploading an XLSX file, we'll need to specify which worksheets to import. This file has the results on sheet 5, so we'll use that. The documentation for the API says this is optional if the import configuration file has a default (which it does) but in practice we need to specify it anyway.
 - `newOrExistingData` - WQX wants to know if we're submitting new data or updating existing data. Usually, you would specify `WQXWeb.CONTAINS_NEW_ONLY` or `WQXWeb.CONTAINS_EXISTING_ONLY`, but in this example we'll use `WQXWeb.CONTAINS_NEW_OR_EXISTING` so we can run the example multiple times without error.
 - `uponCompletion` - There is a more advanced way of monitoring the the multiple steps of processing done by WQX Web, but we're going to skip all that by specifying `WQXWeb.EXPORT_IMPORT` which uses WQX Web to export the XML file needed for direct submission to WQX.
 - `uponCompletionCondition` - WQX Web needs to know what to do if errors or warnings come up while processing your data. We want it to fail if there is even a warning for this example, so we'll use `WQXWeb.EXPORT_IF_NO_WARNING` but you can use the more relaxed `WQXWeb.EXPORT_IF_NO_ERROR` if you want.

`StartImport` will return the ID of the dataset which we will use later to check the status of the import. Let's run it now.

In [6]:
dataset_id = wqxweb.StartImport(
    importConfigurationId="7411",
    fileId=file_id,
    fileType=WQXWeb.XLSX,
    worksheetsToImport="5",
    newOrExistingData=WQXWeb.CONTAINS_NEW_OR_EXISTING,
    uponCompletion=WQXWeb.EXPORT_IMPORT,
    uponCompletionCondition=WQXWeb.EXPORT_IF_NO_WARNING
)
print(f"WQX Web assigned the dataset ID {dataset_id} added the dataset to its work queue.")

WQX Web assigned the dataset ID 72334 added the dataset to its work queue.


To check the status of that dataset, we'll use `GetStatus`. It simply takes a dataset ID and returns a status message in a dictionary object.

That object should contain the following attributes:

 - `StatusType` - A number representing the status type.
 - `StatusName` - A human-readable status message.
 - `StatusUid` - Another number representing the status.
 - `PercentComplete` - A percentage representing how much of the work is completed.
 - `QueuePosition` - If WQX Web is to busy to begin processing your data, it will be in queue and you'll see the queue position here.
 - `TransactionId` - This may be an empty string or a transaction ID.

We're going to call that API endpoint every 10 seconds until it reaches a state that indicates it is no longer busy.

In [7]:
from time import sleep

status = None
busy_states = ('Waiting to Import','Importing','Waiting to Export','Waiting to Export and Submit','Exporting','Processing at CDX','Waiting to Delete','Deleting','Waiting to Update WQX','Updating WQX')

while status is None or status in busy_states:
    status = wqxweb.GetStatus( dataset_id ).get('StatusName')
    if status in busy_states:
        print(f"Current status is '{status}'. Now waiting 10 seconds as requested by the API documentation.")
        sleep(10)
print(f"WQX Web stopped with status: {status}")

WQX Web stopped with status: Exported
